In [1]:
%matplotlib inline

import csv

import scipy
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from IPython.display import FileLink, FileLinks, display, HTML

In [4]:
# Set these options to display all the columns
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [5]:
def write_csv(output_fn, ids, test_y):
    with open(output_fn, "w") as fp:
        writer = csv.writer(fp)
        writer.writerow(['Id', 'SalePrice'])
        for _id, y in zip(ids, test_y):
            writer.writerow([_id, y])

In [6]:
base_path = "./house-prices-advanced-regression-techniques"

In [5]:
testdf = pd.read_csv(f"{base_path}/test.csv")
traindf = pd.read_csv(f"{base_path}/train.csv")

Check for null values and delete them before we proceed. We don't have any in our case

In [57]:
traindf['SalePrice'].isnull().values.any()

False

Understand some basics about the data. For convenience i split them into two parts 

In [59]:
traindf.select_dtypes(exclude=['object']).describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1379.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,567.240411,1057.429452,1162.626712,346.992466,5.844521,1515.463699,0.425342,0.057534,1.565068,0.382877,2.866438,1.046575,6.517808,0.613014,1978.506164,1.767123,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,441.866955,438.705324,386.587738,436.528436,48.623081,525.480383,0.518911,0.238753,0.550916,0.502885,0.815778,0.220338,1.625393,0.644666,24.689725,0.747315,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,223.000000,795.750000,882.000000,0.000000,0.000000,1129.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1961.000000,1.000000,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,477.500000,991.500000,1087.000000,0.000000,0.000000,1464.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,808.000000,1298.250000,1391.250000,728.000000,0.000000,1776.750000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,3.000000,2.000000,3.000000,2.000000,8.000000,3.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [60]:
train_data.select_dtypes(include=['object']).describe()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
count,1460,1460,91,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1452,1460,1460,1460,1423,1423,1422,1423,1422,1460,1460,1460,1459,1460,1460,770,1379,1379,1379,1379,1460,7,281,54,1460,1460
unique,5,2,2,4,4,2,5,3,25,9,8,5,8,6,8,15,16,4,4,5,6,4,4,4,6,6,6,5,2,5,4,7,5,6,3,5,5,3,3,4,4,9,6
top,RL,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,Gable,CompShg,VinylSd,VinylSd,None,TA,TA,PConc,TA,TA,No,Unf,Unf,GasA,Ex,Y,SBrkr,TA,Typ,Gd,Attchd,Unf,TA,TA,Y,Gd,MnPrv,Shed,WD,Normal
freq,1151,1454,50,925,1311,1459,1052,1382,225,1260,1445,1220,726,1141,1434,515,504,864,906,1282,647,649,1311,953,430,1256,1428,741,1365,1334,735,1360,380,870,605,1311,1326,1340,3,157,49,1267,1198


In [7]:
def train_model(train_X, train_y, **kwargs):
    model = XGBRegressor(**kwargs)
    model.fit(train_X, train_y, verbose=True)
    return model

Convert the string data into categorical. 

In [65]:
traindf[traindf.select_dtypes(['object']).columns] = traindf.select_dtypes(['object']).apply(lambda x: x.astype('category'))

In [78]:
y = traindf.SalePrice
X = traindf.drop(['SalePrice'], axis=1).select_dtypes(exclude=['category'])

In [115]:
model = train_model(X, y)

On our first try lets remove all the categorical data and try. 

In [84]:
test_X = testdf.select_dtypes(exclude=['object'])
test_y = model.predict(test_X)

In [108]:
write_csv("house_price_predict_1.csv", test_X['Id'], test_y)
FileLink("house_price_predict_1.csv")

/media/disk6tb/myfiles/fastai_v1/notebooks/house_price_predict_1.csv

After submission, we got about 0.14179 score with our test data. We are approximately at 50 percentile ! position. You might get a different score because of randomization. 

In [119]:
X1 = traindf.drop(['SalePrice'], axis=1)
X1 = pd.get_dummies(X1)
test_X1 = pd.get_dummies(testdf)

In [120]:
model_cat = train_model(X1, y)
model_cat.predict(test_X1)

ValueError: feature_names mismatch: ['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold'] ['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenP

In [125]:
train_total = len(traindf)
df_concat = pd.get_dummies(pd.concat([traindf.drop(['SalePrice'], axis=1), testdf], axis=0))
train_dummy = df_concat[:train_total]
test_dummy = df_concat[train_total:]

In [128]:
model_dummy = train_model(train_dummy, y)
y_dummy = model_dummy.predict(test_dummy)

In [129]:
write_csv("house_price_predict_2.csv", test_dummy['Id'], y_dummy)
FileLink("house_price_predict_2.csv")

/media/disk6tb/myfiles/fastai_v1/notebooks/house_price_predict_2.csv

In [130]:
model_dummy

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

Got about 0.136 which is some improvement over previous value

In [134]:
traindf.select_dtypes(['category']).columns

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition'], dtype='object')

In [151]:
df_label = pd.concat([traindf.drop(['SalePrice'], axis=1), testdf], axis=0)
df_label[df_label.select_dtypes(['object']).columns] = df_label.select_dtypes(['object']).apply(lambda x: x.astype('category'))
cat_columns = df_label.select_dtypes(['category']).columns
df_label[cat_columns] = df_label[cat_columns].apply(lambda x: x.cat.codes)
train_label = df_label[:train_total]
test_label = df_label[train_total:]

In [153]:
train_label.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
  

In [154]:
test_label.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
  

In [158]:
model_label = train_model(train_label, y)
y_label = model_label.predict(test_label)
write_csv("house_price_predict_3.csv", test_label['Id'], y_label)
FileLink("house_price_predict_3.csv")

/media/disk6tb/myfiles/fastai_v1/notebooks/house_price_predict_3.csv

Got 0.13793 not an improvement with the previous numbers

In [165]:
base_score = np.median(y)

model_dummy = train_model(train_dummy, y, base_score=base_score, n_estimators=200, max_depth=4,learning_rate=0.01)
y_dummy = model_dummy.predict(test_dummy)
write_csv("house_price_predict_4.csv", test_dummy['Id'], y_dummy)
FileLink("house_price_predict_4.csv")

/media/disk6tb/myfiles/fastai_v1/notebooks/house_price_predict_4.csv

In [166]:
model_label = train_model(train_label, y, base_score=base_score, n_estimators=200, max_depth=4,learning_rate=0.01)
y_label = model_label.predict(test_label)
write_csv("house_price_predict_5.csv", test_label['Id'], y_label)
FileLink("house_price_predict_5.csv")

/media/disk6tb/myfiles/fastai_v1/notebooks/house_price_predict_5.csv

In [12]:
traindf.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
  

In [21]:
base_score = np.median(y)

cols_to_ignore = ["PoolQC", "MiscFeature", "Alley", "Fence", "FireplaceQu", 
                  "LotFrontage", "GarageCond", "GarageType", "GarageYrBlt",
                  "GarageFinish", "GarageQual", "BsmtExposure", "BsmtFinType2",
                  "BsmtFinType1", "BsmtCond", "BsmtQual", "MasVnrArea", "MasVnrType"]
y = traindf.SalePrice

train_total = len(traindf)
df_concat = pd.concat([traindf.drop(["SalePrice"], axis=1), testdf], axis=0)
df_concat = df_concat.drop(cols_to_ignore, axis=1)
df_concat = pd.get_dummies(df_concat)
train_dummy = df_concat[:train_total]
test_dummy = df_concat[train_total:]

model_dummy = train_model(train_dummy, y, base_score=base_score, n_estimators=400, max_depth=4,learning_rate=0.05,  gamma=0)
y_dummy = model_dummy.predict(test_dummy)

write_csv("house_price_ignore_dummies.csv", test_dummy['Id'], y_dummy)
FileLink("house_price_ignore_dummies.csv")

/media/disk6tb/myfiles/fastai_v1/notebooks/house_price_ignore_dummies.csv

In [51]:
base_score = np.median(y)

cols_to_ignore = ["PoolQC", "MiscFeature", "Alley", "Fence", "FireplaceQu", 
                  "LotFrontage", "GarageCond", "GarageType", "GarageYrBlt",
                  "GarageFinish", "GarageQual", "BsmtExposure", "BsmtFinType2",
                  "BsmtFinType1", "BsmtCond", "BsmtQual", "MasVnrArea", "MasVnrType"]

# Remove outliers. 
df_train = traindf.drop(traindf[traindf['Id'] == 1299].index)
df_train = df_train.drop(df_train[df_train['Id'] == 524].index)

y = df_train.SalePrice

train_total = len(df_train)

df_concat = pd.concat([df_train.drop(['SalePrice'], axis=1), testdf], axis=0)
df_concat = df_concat.drop(cols_to_ignore, axis=1)

# Add transformations here. 
y_log = np.log(y)
df_concat['GrLivArea'] = np.log(df_concat['GrLivArea'])

df_concat['HasBsmt'] = pd.Series(len(df_concat['TotalBsmtSF']), index=df_concat.index)
df_concat['HasBsmt'] = 0 
df_concat.loc[df_concat['TotalBsmtSF']>0,'HasBsmt'] = 1
df_concat['TotalBsmtSF'] = np.where(df_concat['TotalBsmtSF']>0, np.log(df_concat['TotalBsmtSF']), 0)
#df_concat.loc[df_concat['TotalBsmtSF']>0,'TotalBsmtSF'] = np.log(df_concat['TotalBsmtSF'])

df_concat = pd.get_dummies(df_concat)


train_dummy = df_concat[:train_total]
test_dummy = df_concat[train_total:]

model_dummy = train_model(train_dummy, y_log, base_score=base_score, n_estimators=700, max_depth=4, learning_rate=0.05, gamma=0)
y_dummy = model_dummy.predict(test_dummy)
y_pred = np.exp(y_dummy)

write_csv("house_price_ignore_dummies_trans.csv", test_dummy['Id'], y_pred)
FileLink("house_price_ignore_dummies_trans.csv")

/home/sairahul/.local/share/virtualenvs/fastai_v1-1ZccQiGW/lib/python3.6/site-packages/ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in log


/media/disk6tb/myfiles/fastai_v1/notebooks/house_price_ignore_dummies_trans.csv

In [35]:
print(df_concat.shape)
print(len(traindf))
print(len(testdf))
print(len(traindf)+len(testdf))

(2917, 221)
1460
1459
2919


In [46]:
print(y_pred[:20])
print(y_dummy[:20])

[132836.6  167921.19 190279.62 190279.62 222459.42 175979.89 187329.61
 167921.19 208981.28 130777.15 215615.06 105082.39 108418.06 167921.19
 150523.58 390428.44 256049.23 281215.06 276855.22 456457.1 ]
[11.796875 12.03125  12.15625  12.15625  12.3125   12.078125 12.140625
 12.03125  12.25     11.78125  12.28125  11.5625   11.59375  12.03125
 11.921875 12.875    12.453125 12.546875 12.53125  13.03125 ]


In [144]:
def show_missing_data(df):
    total = df.isnull().sum().sort_values(ascending=False)
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1)
    display(missing_data.head(30))

show_missing_data(traindf)
show_missing_data(testdf)

,0,1
PoolQC,1453,0.995205
MiscFeature,1406,0.963014
Alley,1369,0.937671
Fence,1179,0.807534
FireplaceQu,690,0.472603
LotFrontage,259,0.177397
GarageCond,81,0.055479
GarageType,81,0.055479
GarageYrBlt,81,0.055479
GarageFinish,81,0.055479


,0,1
PoolQC,1456,0.997944
MiscFeature,1408,0.965045
Alley,1352,0.926662
Fence,1169,0.801234
FireplaceQu,730,0.500343
LotFrontage,227,0.155586
GarageCond,78,0.053461
GarageQual,78,0.053461
GarageYrBlt,78,0.053461
GarageFinish,78,0.053461


You can see that data is missing in some columns. These columns are different for test vs train. We need to come up with some strategy to replace the null fields. For convenience we will merge the train and test sets together and apply the same transformations for both the datasets.

In [32]:
testdf = pd.read_csv(f"{base_path}/test.csv")
traindf = pd.read_csv(f"{base_path}/train.csv")

y = traindf.SalePrice
df = pd.concat([traindf.drop(['SalePrice'], axis=1), testdf], axis=0)

In [141]:
neighborhood_lot = dict(df.groupby('Neighborhood')['LotFrontage'].mean())
df['LotFrontage'] = df.apply(lambda row: row['LotFrontage'] if pd.notnull(row['LotFrontage']) else neighborhood_lot[row['Neighborhood']], axis=1)
df['GarageYrBlt'] = df.apply(lambda row: row['GarageYrBlt'] if pd.notnull(row['GarageYrBlt']) else row['YearBuilt'])

In [7]:
#from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from category_encoders.ordinal import OrdinalEncoder

rating_mapping = [
    ("Ex",5), 
    ("Gd",4), 
    ("TA",3), 
    ("Fa",2), 
    ("Po",1),
    (np.NaN,0),
]

basement_type = [
    ("GLQ", 6),
    ("ALQ", 5),
    ("BLQ", 4),
    ("Rec", 3),
    ("LwQ", 2),
    ("Unf", 1),
    (np.NaN, 0),            
]

cols_mapping = [
    {
        "col": "PoolQC",    
        "mapping": rating
    },
    {
        "col": "FireplaceQu",
        "mapping": rating
    },
    {
        "col": "LotShape",
        "mapping": [
            ("Reg", 3),
            ("IR1", 2),
            ("IR2", 1),
            ("IR3", 0),
        ]
    },
    {
        "col": "GarageCond",
        "mapping": rating
    },
    {
        "col": "GarageQual",
        "mapping": rating
    },
    {
        "col": "BsmtQual",
        "mapping": rating
    },
    {
        "col": "BsmtCond",
        "mapping": rating
    },
    {
        "col": "BsmtExposure",
        "mapping": rating
    },
    {
        "col": "BsmtFinType1",
        "mapping": basement_type
    },
    {
        "col": "BsmtFinType2",
        "mapping": basement_type
    },
    {
        "col": "BsmtQual",
        "mapping": basement_type
    },
    {
        "col": "MasVnrType",
        "mapping": [
            ("Stone", 2),
            ("BrkFace", 1),
            ("BrkCmn", 0),
            (np.NaN, 0),
        ]
    },
    {
        "col": "KitchenQual",
        "mapping": rating
    },
    {
        "col": "Functional",
        "mapping": [
            (np.NaN, 0),
            ("Sal", 0), 
            ("Sev", 1), 
            ("Maj2", 2),
            ("Maj1", 3),
            ("Mod", 4),
            ("Min2", 5),
            ("Min1", 6),
            ("Typ", 7),
        ]
    },
    {
        "col": "ExterQual",
        "mapping": rating
    },
    {
        "col": "ExterCond",
        "mapping": rating
    },    
]

cols_to_drop = ['MiscFeature', 'Utilities']
cols_category =  ['MiscFeature', 'Alley', 'Fence', 'GarageFinish', 'LotConfig', 'GarageType', 'MSZoning', 'Exterior1st', 'Exterior2nd', 'Electrical', 'SaleType', 'SaleCondition']
cols_ordinal = [row['col'] for row in cols_mapping]
cols_numeric = ['BsmtFullBath', 'BsmtHalfBath', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'MasVnrArea']

col_trans = make_column_transformer(
    (
        OrdinalEncoder(mapping=cols_mapping, return_df=True), cols_ordinal
    ),
    (
        Pipeline([
            ('si', SimpleImputer(strategy='constant', fill_value='NA')),
            ('oe', OneHotEncoder(sparse=False))
        ]),
        cols_cat
    ),
    (
        SimpleImputer(strategy='constant', fill_value=0), cols_numeric
    ),
    remainder='passthrough')



In [161]:
#GarageCond	78	0.053461
#GarageQual	78	0.053461
#GarageYrBlt	78	0.053461
#GarageFinish	78	0.053461
#GarageType	76	0.052090
df['Functional'].value_counts()


Typ     2717
Min2      70
Min1      65
Mod       35
Maj1      19
Maj2       9
Sev        2
Name: Functional, dtype: int64

In [9]:

aa = col_trans.fit_transform(df)
#aa = pd.DataFrame(aa, columns=df.columns)

In [69]:
# Label encoding
??OrdinalEncoder

1. http://fastml.com/how-to-use-pd-dot-get-dummies-with-the-test-set/
1. https://stackoverflow.com/questions/41335718/keep-same-dummy-variable-in-training-and-testing-data
1. https://stackoverflow.com/questions/32011359/convert-categorical-data-in-pandas-dataframe
1. https://stackoverflow.com/questions/24458645/label-encoding-across-multiple-columns-in-scikit-learn
1. https://www.kaggle.com/pmarcelino/comprehensive-data-exploration-with-python 
1. https://www.kaggle.com/erikbruin/house-prices-lasso-xgboost-and-a-detailed-eda